# 5-point Relative Pose Problem
## Server
This notebook provides the server-side of the **5-point relative pose** application. The purpose of this application is to identify the possible relative camera motions given five matching points from two calibrated views. 
The image below provides a visualization of such problem.

![5-point_relpose](img/5-point_relpose.png "5-point Relative Pose Problem")

Refer to the [introduction notebook](../introduction.ipynb) for further info on the project and the available notebooks.

### Constants definition
First, we need to define some global variables that specify parameters of the accelerator.

In [ ]:
_max_input_points = 1024
_num_random_numbers = 1000000
_rand_max = 2147483647 # from stdlib.h
_max_ransac_iter = 1024*16
_points_per_iter = 5
_extra_ransac_points = 3
_total_points_per_iter = _points_per_iter + _extra_ransac_points
_nister_fpga_max_out_shape = (_max_ransac_iter,10,10)
_nister_fgpa_max_in_shape = (_max_ransac_iter,_points_per_iter,2)

### Download Overlay and Buffers Instantiation
We can now import `pynq`, download the overlay, and assign the 5-point kernel IP to a variable called `fivept_nister`, that we will use later on. We also allocate the required input and output buffers using the `pynq.allocate`.

In [ ]:
import pynq
import numpy as np

ol = pynq.Overlay("5point_nister.xclbin")
fivept_nister = ol.fivept_nister_1

input_a = pynq.allocate(_nister_fgpa_max_in_shape, dtype=np.float32)
input_b = pynq.allocate(_nister_fgpa_max_in_shape, dtype=np.float32)
out_buff = pynq.allocate(_nister_fpga_max_out_shape, dtype=np.float32)

### Points pre-processing and 5-point Nister software implementation

The cell below defines the `gen_5_point_problems()` and `multi_nister_sw()` functions.

The first function generates the 5-point problems that either the CPU or the FPGA are going to solve. It constructs two vectors of points randomly extracted from the two sources, according to the given number of problems. The original C++ implementation can be found [here](https://bitbucket.org/necst/xohw18_5points_public/src/master/final-demo/server/src/ransac.cpp#lines-11).

The second function computes the 5-point [Nister's algorithm](https://ieeexplore.ieee.org/abstract/document/1288525) on the input points and produces a list of [essential matrices](https://en.wikipedia.org/wiki/Essential_matrix) as a result, using the CPU. The function relies on [OpenGV](https://laurentkneip.github.io/opengv/) and it's python bindings to perform the core computation. The original C++ implementation can be found [here](https://bitbucket.org/necst/xohw18_5points_public/src/master/final-demo/server/src/nister_sw.cpp).

In both cases, we use numpy's features to obtain code that is both compact and fast.

In [ ]:
from fivepoint_pynq import pyopengv, ransac

def gen_5_point_problems(p1_in, p2_in, n_points, p1_out, p2_out, n_problems,
                         out_indices, rand_numbers, rand_offset):
    if n_points < _total_points_per_iter:
        raise Exception('ransac: required at least {} but only {} points provided'.format(_total_points_per_iter,
                                                                                          n_points))
    tmp_indices = np.arange(n_points)
    j = np.tile(np.arange(_total_points_per_iter), n_problems)
    rand_offsets = np.arange(rand_offset, rand_offset + n_problems*_total_points_per_iter) % rand_numbers.size
    rand_positions = rand_numbers[rand_offsets] % (n_points - j) + j 
    p1_out[:_points_per_iter*n_problems] = \
        p1_in[rand_positions.reshape((n_problems,_total_points_per_iter))[:,:_points_per_iter].reshape(-1)]
    p2_out[:_points_per_iter*n_problems] = \
        p2_in[rand_positions.reshape((n_problems,_total_points_per_iter))[:,:_points_per_iter].reshape(-1)]
    out_indices[:n_problems*_total_points_per_iter] = tmp_indices[rand_positions]
    return rand_offsets[-1]

def multi_nister_sw(p1, p2, num_iter, essentials, essential_sizes):
    b2 = np.empty((num_iter, _points_per_iter, 3))
    b1 = np.empty((num_iter, _points_per_iter, 3))
    b1[:,:,0:2] = p1[:num_iter*_points_per_iter].reshape((num_iter,_points_per_iter,2))
    b1[:,:,2] = 1.0
    b2[:,:,0:2] = p2[:num_iter*_points_per_iter].reshape((num_iter,_points_per_iter,2))
    b2[:,:,2] = 1.0
    b1[:,:] = b1[:] / np.linalg.norm(b1[:], axis=2).reshape(num_iter,_points_per_iter, 1)
    b2[:,:] = b2[:] / np.linalg.norm(b2[:], axis=2).reshape(num_iter,_points_per_iter, 1)
    for i in range(num_iter):
        ess = pyopengv.relative_pose_fivept_nister(b1[i], b2[i])
        len_ess = len(ess)
        if len_ess > 0:
            essentials[essential_sizes[i]:essential_sizes[i]+len_ess] = ess
        essential_sizes[1 + i] = essential_sizes[i] + len_ess

### 5-point Nister FPGA implementation
We now define the `multi_nister_fpga()` function, that executes the 5-point Nister's algorithm on the FPGA. Notice the usual pattern of calls to execute an accelerator using PYNQ. First, we `sync_to_device()` the input buffers - after copying the input data from `p1` and `p2` - then we `call()` the accelerator and then we `sync_from_device()` the output buffer to retrieve the results, that we use to populate `essentials`.

Remember that you can print `fivept_nister.signature` to retrieve the signature of the `call()` function.

The original C++ implementation, that relies on OpenCL, can be found [here](https://bitbucket.org/necst/xohw18_5points_public/src/master/final-demo/server/src/nister_fpga.cpp).

In [ ]:
def multi_nister_fpga(p1, p2, num_iter, essentials, essential_sizes, fivept_nister_krnl=fivept_nister, 
                      input_a=input_a, input_b=input_b, out_buff=out_buff):
    threshold = np.float32(0.01)
    input_a[:num_iter,:,:] = p1.reshape((-1,_points_per_iter,2))[:num_iter,:,:]
    input_b[:num_iter,:,:] = p2.reshape((-1,_points_per_iter,2))[:num_iter,:,:]
    input_a.sync_to_device()
    input_b.sync_to_device()
    fivept_nister_krnl.call(input_a, input_b, out_buff, num_iter, threshold)
    out_buff.sync_from_device()
    tmp_essentials = out_buff[:num_iter,:,:]
    tmp_essentials_valid_count = np.cumsum(np.sum(tmp_essentials[:,:,0] > 0,axis=1))
    # each essential_t matrix (a 3x3) is stored column-major. We use '[0,3,6,1,4,7,2,5,8]' to reorder to column-major.
    tmp_essentials = tmp_essentials[tmp_essentials[:,:,0] > 0,1:10][:,[0,3,6,1,4,7,2,5,8]].reshape(-1,3,3)
    essentials[:len(tmp_essentials)] = tmp_essentials[:]
    essential_sizes[1:] = tmp_essentials_valid_count[:]

### Server Handler

The logic to run the server application is defined in the following cell. We use the [`websockets`](https://websockets.readthedocs.io/en/stable/intro.html) and [`asyncio`](https://docs.python.org/3/library/asyncio.html) python packages to communicate with the client. The server parses requests from the client and sends the responses in the `consumer_handler()` function.

As we were not able to reimplement the `ransac()` function using Python (due to the limited python bindings offered by OpenGV, and most importantly because of the changes made in the original Xilinx Open Hardware project with respect to the original OpenGV RANSAC) we rely on [pybind11](https://github.com/pybind/pybind11) to expose python bindings for this function.

The original C++ implementation of the `main()` function can be found [here](https://bitbucket.org/necst/xohw18_5points_public/src/master/final-demo/server/main.cpp), while the `ransac()` implementation can be found [here](https://bitbucket.org/necst/xohw18_5points_public/src/master/final-demo/server/src/ransac.cpp#lines-261).

In [ ]:
import asyncio
import websockets
import time 

np.set_printoptions(precision=16)
        
async def consumer_handler(websocket, message, p1, p2, p1_ransac, p2_ransac, indices,
                           random_numbers, rand_offset):
    msg = message.split()

    # read input message
    message_id = msg[0]
    n_points = int(msg[1])
    print('number of points: {}'.format(n_points))
    if n_points > _max_input_points:
        raise valueError('number of input points ({}) exceeds the maximum allowed ({})'.format(
            n_points, _max_input_points))
    elif n_points == 0:
        raise ValueError('number of input points is 0')
    algorithm_num = int(msg[2])
    print('algorithm: {}'.format(algorithm_num))
    if algorithm_num not in [0,1]:
        raise ValueError('invalid 5-point algorithm')
    num_ransac_iter = int(msg[3])
    print('RANSAC iterations: {}'.format(num_ransac_iter))
    if num_ransac_iter > _max_ransac_iter:
        raise ValueError('number of ransac iterations ({}) exceeds the maximum allowed ({})'.format(
            num_ransac_iter, _max_ransac_iter))
    ransac_threshold = float(msg[4])
    msg_idx = 5
    p1.reshape(-1)[:n_points*2] = [*map(float, msg[msg_idx:msg_idx + n_points*2])]
    p2.reshape(-1)[:n_points*2] = [*map(float, msg[msg_idx + n_points*2:msg_idx + n_points*2*2])]
    transformation = np.empty((3,4))
    essentials = np.zeros((num_ransac_iter*10, 3, 3))
    essential_sizes = np.zeros(num_ransac_iter+1, dtype=np.uint32)

    # generate 5-point problems to solve
    start = time.process_time()
    rand_offset = gen_5_point_problems(p1, p2, n_points, p1_ransac, p2_ransac, num_ransac_iter, 
                                       indices, random_numbers, rand_offset)
    generation_time = time.process_time() - start

    # solve 5-point problem using either CPU or FPGA
    ## 1 - retrieve essential matrices
    if algorithm_num == 0: # Software execution
        multi_nister_sw(p1_ransac, p2_ransac, num_ransac_iter, essentials,
                        essential_sizes)
    elif algorithm_num == 1: # FPGA execution
        multi_nister_fpga(p1_ransac, p2_ransac, num_ransac_iter, essentials, 
                          essential_sizes)
    essentials_time = time.process_time() - generation_time - start
    ## 2 - estimate the best transformation
    quality = ransac(essentials, essential_sizes, num_ransac_iter, indices, 
                     p1, p2, n_points, transformation, ransac_threshold)
    recovery_time = time.process_time() - essentials_time - generation_time - start

    # send output message with results
    out_msg = message_id + ' '
    out_msg += ' '.join(map(str, transformation[:,:3].flatten())) + ' '
    out_msg += ' '.join(map(str, transformation[:,3])) + ' '
    out_msg += '{} {} {} {}\n'.format(recovery_time, essentials_time, generation_time, quality)
    await websocket.send(out_msg)
    print('best transformation recovery time: {}'.format(recovery_time))
    print('essentials computation time: {}'.format(essentials_time))
    print('ransac generation time: {}'.format(generation_time))

async def server(websocket, path):
    print("connection established...")
    msg_queue = asyncio.Queue()

    # Allocate required arrays and variables
    p1 = np.empty((_max_input_points, 2), dtype=np.float32)
    p2 = np.empty((_max_input_points, 2), dtype=np.float32)
    p1_ransac = np.empty((_max_ransac_iter*_points_per_iter, 2), dtype=np.float32)
    p2_ransac = np.empty((_max_ransac_iter*_points_per_iter, 2), dtype=np.float32)
    indices = np.empty(_max_ransac_iter*_total_points_per_iter, dtype=np.uint32)
    random_numbers = np.random.randint(0, _rand_max, _num_random_numbers)
    rand_offset = 0

    print('ready...')
    await websocket.send('READY')
    # Start execution
    async for message in websocket:
        await consumer_handler(websocket, message, p1, p2, p1_ransac, p2_ransac, 
                               indices, random_numbers, rand_offset)
    print('terminating...')

### Run the Server
We can now run the server and make it listen for requests on `ws://localhost:8080`. Remember that you will need to manually interrupt the kernel to stop the server execution. Change `host` address and `port` appropriately in case you want to enable remote access to the websocket.

In [ ]:
import nest_asyncio

ws_server = websockets.serve(server, host='localhost', port=8080, ping_timeout=60, close_timeout=30)
loop = asyncio.get_event_loop()
nest_asyncio.apply(loop=loop)
loop.run_until_complete(ws_server)
loop.run_forever()

### Close Websocket
In case needed, stop the execution of the cell above and `close()` the websocket to free up the used port so you can launch the server again if you need to perform some modifications. Otherwise, you will have to restart the IPython kernel.

In [ ]:
ws_server.ws_server.close()

### Clean FPGA resources

When done, you can free the FPGA resources by shutting down this notebook or executing the following cell

In [ ]:
del input_a
del input_b
del out_buff
ol.free()

Copyright (C) 2020 Xilinx, Inc